## Compressione

Si procede in primo luogo a riorganizzre il dataset Twitch in maniera tale che ogni gioco sia rappresentato da un solo documento. Questo costituisce un vantaggio in fase di integrazione. Infatti, in questo modo, la medesima informazione (ad esempio i generi del gioco) è rappresentata una ed una sola volta, senza creare una ridondanza inutile e fine a se stessa.

In [1]:
import json
import pandas as pd
import re
from unidecode import unidecode

In [2]:
Twitch = pd.read_json(r"C:\Users\39331\Desktop\dataset_twitch.json")

In [3]:
Giantbomb = pd.read_json(r"C:\Users\39331\Desktop\Giantbomb.json")

Si procede ad effettuare la conversione del tipo degli elementi delle colonne *viewers*,*giantbomb_id* e *timestamp*, in quanto la libreria *JSON* non è in grado di gestire i dati di tipo int64 e timestamp nel momento in cui deve esportare un json. 

In [4]:
Twitch['viewers'] = Twitch['viewers'].apply(lambda x: float(x))
Twitch['giantbomb_id'] = Twitch['giantbomb_id'].apply(lambda x: str(x))
Twitch['timestamp'] = Twitch['timestamp'].apply(lambda x: str(x))

Si procede a creare una funzione che, dato in input un pandas Dataframe contenente i records che si riferiscono ad un determinato gioco, restituisce un dizionario così strutturato:

* Una coppia chiave-valore per gestire il nome del gioco;
* Una coppia chiave-valore per gestire l'id di Giantbomb del gioco;
* Una coppia chiave-valore, dove il valore è una lista contenente una serie di dizionari, con chiave il timestamp e valore il numero di spettatori rilevati in quell'istante temporale.

In [5]:
def create_diz(df):
    df = df.reset_index()
    diz = {}
    diz['name'] = df['name'].unique()[0]
    diz['giantbomb_id'] = df['giantbomb_id'].unique()[0]
    views = []
    for i in range(0, df.shape[0]):
        views.append({'timestamp': df['timestamp'][i],'visuals':df['viewers'][i],
                     'ranking':float(df['ranking'][i])})
    diz['views'] = views
    return(diz)

In [6]:
nomi = Twitch.name.unique()

Si itera lungo la lista *nomi*, la quale contiene il nome di ogni gioco presente nel dataset *Twitch*, senza duplicati. Ad ogni iterazione, si inserisce dentro la lista un dizionario (che rappresenta un gioco), creato secondo la funzione descritta in precedenza.

In [7]:
lista = []
for i in nomi:
    lista.append(create_diz(Twitch[Twitch['name'] == i]))

Per completezza, si riporta un esempio del risultato della procedura di "compressione".

In [8]:
lista[0]

{'name': 'Fortnite',
 'giantbomb_id': '37030',
 'views': [{'timestamp': '2020-07-02 08:30:53',
   'visuals': 116640.0,
   'ranking': 1.0},
  {'timestamp': '2020-07-02 08:31:53', 'visuals': 116640.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:32:54', 'visuals': 117569.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:33:54', 'visuals': 117569.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:34:54', 'visuals': 116580.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:35:55', 'visuals': 116580.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:36:55', 'visuals': 116580.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:37:56', 'visuals': 117772.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:38:56', 'visuals': 117772.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:39:56', 'visuals': 116520.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:40:57', 'visuals': 116520.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:41:57', 'visuals': 117314.0, 'ranking': 1.0},
  {'timestamp'

Si procede all'esportazione in json.

In [9]:
with open(r"C:\Users\39331\Desktop\Twitch_compresso.json", "w") as f:
       json.dump(lista, f)

## Aggregazione

Ora a partire dal dataset "Twitch_compresso", si procede ad effettuare l'integrazione con il dataset di Giantbomb. L'integrazione in questo caso è molto semplice dal momento che i due dataset dialogano tra di loro tramite una chiave costituta dall'attributo *giantbomb_id*. Pertanto, è sufficiente effettuare un inner join tra i due dataset, dove la chiave esterna è *giantbomb_id*.

In [10]:
Twitch_compresso = pd.read_json(r"C:\Users\39331\Desktop\Twitch_compresso.json")

In [11]:
Twitch_compresso.head()

,name,giantbomb_id,views
0,Fortnite,37030,"[{'timestamp': '2020-07-02 08:30:53', 'visuals..."
1,League of Legends,24024,"[{'timestamp': '2020-07-02 08:30:53', 'visuals..."
2,VALORANT,77445,"[{'timestamp': '2020-07-02 08:30:53', 'visuals..."
3,Call of Duty: Modern Warfare,73517,"[{'timestamp': '2020-07-02 08:30:53', 'visuals..."
4,Counter-Strike: Global Offensive,36113,"[{'timestamp': '2020-07-02 08:30:53', 'visuals..."


In [12]:
Twitch_Giantbomb = pd.merge(Twitch_compresso, Giantbomb, on = "giantbomb_id")

In [13]:
Twitch_Giantbomb = Twitch_Giantbomb.drop(columns = ['name_x']) 

In [14]:
Twitch_Giantbomb = Twitch_Giantbomb.rename(columns = {'name_y':'name'})

In [18]:
Twitch_Giantbomb.head()

,giantbomb_id,views,name,original_release_date,platforms,characters,developers,genres,similar_games,themes,dlcs,norm_name
0,37030,"[{'timestamp': '2020-07-02 08:30:53', 'visuals...",Fortnite,2017-07-25,"[{'name': 'Mac'}, {'name': 'PC'}, {'name': 'iP...","[{'name': 'Batman'}, {'name': 'Catwoman'}, {'n...","[{'date_founded': '1991-01-01', 'developed_gam...",[{'name': 'Shooter'}],"[{'name': 'CastleMiner'}, {'name': 'Minecraft'...","[{'name': 'Horror'}, {'name': 'Fantasy'}, {'na...",NaN,fortnite
1,24024,"[{'timestamp': '2020-07-02 08:30:53', 'visuals...",League of Legends,2009-10-27,"[{'name': 'Mac'}, {'name': 'PC'}]","[{'name': 'Annie, the Dark Child'}, {'name': '...","[{'date_founded': '2006-01-01', 'developed_gam...",[{'name': 'MOBA'}],"[{'name': 'Heroes of Newerth'}, {'name': 'Infi...",[{'name': 'Fantasy'}],NaN,leagueoflegends
2,77445,"[{'timestamp': '2020-07-02 08:30:53', 'visuals...",Valorant,2020-06-02,[{'name': 'PC'}],"[{'name': 'Phoenix'}, {'name': 'Jett'}, {'name...","[{'date_founded': '2006-01-01', 'developed_gam...",[{'name': 'First-Person Shooter'}],"[{'name': 'Team Fortress 2'}, {'name': 'Overwa...",[{'name': 'Sci-Fi'}],NaN,valorant
3,73517,"[{'timestamp': '2020-07-02 08:30:53', 'visuals...",Call of Duty: Modern Warfare,2019-10-25,"[{'name': 'PC'}, {'name': 'Xbox One'}, {'name'...","[{'name': 'Captain Price'}, {'name': 'Gaz'}, {...","[{'date_founded': '2002-01-01', 'developed_gam...",[{'name': 'First-Person Shooter'}],None,[{'name': 'Modern Military'}],NaN,callofdutymodernwarfare
4,36113,"[{'timestamp': '2020-07-02 08:30:53', 'visuals...",Counter-Strike: Global Offensive,2012-08-21,"[{'name': 'Mac'}, {'name': 'Xbox 360 Games Sto...",None,"[{'date_founded': '2006-01-01', 'developed_gam...",[{'name': 'First-Person Shooter'}],"[{'name': 'Call of Duty: Modern Warfare 3'}, {...",[{'name': 'Modern Military'}],"[{'name': 'Operation Payback'}, {'name': 'Oper...",counterstrikeglobaloffensive


## Normalizzazione

Per favorire l'integrazione con igdb, si procede a creare un attributo chiamato "norm_name", il quale contiene il nome normalizzato, ossia senza spazi bianchi, caratteri speciali e lettere maiuscole.

In [16]:
def clean_string(string):
    string = string.replace(" ", "") #rimuove spazi bianchi
    string = string.lower() #si trasformano le lettere minuscole in lettere maiscole
    string = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", string) #rimuovere caratteri speciali
    string = unidecode(string) #trasforma da Unicode ad ASCII
    return string

In [17]:
Twitch_Giantbomb['norm_name'] = Twitch_Giantbomb['name'].apply(clean_string)

Si procede ad esportare il dataset in json

In [19]:
Twitch_Giantbomb.to_json(r"C:\Users\39331\Desktop\Twitch_Giantbomb.json", orient = "records", lines = True)

Infine, si procede a verificare se l'esportazione in json è andata a buon fine. 
E necessario leggerlo in questa maniera in quanto json.load non è in grado di leggere nested json.

In [20]:
games = []
for line in open(r'C:\Users\39331\Desktop\Twitch_Giantbomb.json', 'r'):
    games.append(json.loads(line))

In [21]:
games[0]

{'giantbomb_id': 37030,
 'views': [{'timestamp': '2020-07-02 08:30:53',
   'visuals': 116640.0,
   'ranking': 1.0},
  {'timestamp': '2020-07-02 08:31:53', 'visuals': 116640.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:32:54', 'visuals': 117569.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:33:54', 'visuals': 117569.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:34:54', 'visuals': 116580.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:35:55', 'visuals': 116580.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:36:55', 'visuals': 116580.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:37:56', 'visuals': 117772.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:38:56', 'visuals': 117772.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:39:56', 'visuals': 116520.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:40:57', 'visuals': 116520.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:41:57', 'visuals': 117314.0, 'ranking': 1.0},
  {'timestamp': '2020-07-02 08:42:58'

Il risultato ottenuto è quanto desiderato. Come ultimo controllo, si procede a verificare quanti sono i giochi andati persi durante questa prima fase di integrazione. Per fare ciò è sufficiente confrontare la lunghezza della lista *games* (lista ottenuta dopo il processo di integrazione) e la lista *lista*.

In [22]:
print('lunghezza lista: ',len(lista))
print('lunghezza games: ', len(games))

lunghezza lista:  950
lunghezza games:  939


Si osserva che 11 giochi sono stati persi in seguito al processo di integrazione. Per fortuna, sono giochi poco rilevanti, quindi ai fini dell'analisi non così fondamentali. 